In [42]:
# Sklearn imports
from sklearn.model_selection import train_test_split, KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC, LinearSVC
from sklearn.pipeline import make_pipeline
from scipy.sparse import csr_matrix
from sklearn.neural_network import MLPClassifier

# text libs
from sklearn.feature_extraction.text import TfidfVectorizer

import lime
import lime.lime_text
import lime.lime_tabular
import numpy as np
import pandas as pd
from tqdm import tqdm
from scipy import spatial, stats
import matplotlib.pyplot as plt
import os
import re
import seaborn as sns
from enum import Enum
import random

In [43]:
import copy
import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn import functional as F
from torch.utils.data import DataLoader
from folktables import ACSDataSource, ACSIncome

# local libraries
import exp_utils as eu

In [7]:
%load_ext autoreload
%autoreload 2
%load_ext lab_black

In [44]:
# dataset selection
class Data(Enum):
    REVIEWS = 1
    INCOME = 2


Data = Enum("Data", ["REVIEWS", "INCOME"])

In [61]:
# set dataset
# dataset = Data.REVIEWS
DS = Data.INCOME
dataset_str = str(DS).split(".")[1]

In [47]:
if DS == Data.REVIEWS:
    train_df = pd.read_csv("data/reviews_train.csv")
    test_df = pd.read_csv("data/reviews_test.csv")

    tf_vec = TfidfVectorizer(lowercase=False, stop_words="english")
    X_train_vec = tf_vec.fit_transform(train_df["text"].values).toarray()
    X_test_vec = tf_vec.transform(test_df["text"].values).toarray()
    X_train = train_df["text"].values
    X_test = test_df["text"].values
    y_train = train_df["label"]
    y_test = test_df["label"]

elif DS == Data.INCOME:
    state = "HI"
    year = "2018"
    data_source = ACSDataSource(survey_year=year, horizon="1-Year", survey="person")
    acs_data = data_source.get_data(states=[state], download=True)
    features, label, group = ACSIncome.df_to_numpy(acs_data)
    X_train, X_test, y_train, y_test = train_test_split(
        features, label, test_size=0.20, random_state=42
    )
    scale = StandardScaler()
    X_train_vec = scale.fit_transform(X_train)
    X_test_vec = scale.transform(X_test)


t_xtrain = torch.tensor(X_train_vec, dtype=torch.float32)
t_ytrain = torch.tensor(y_train, dtype=torch.float32).reshape(-1, 1)
t_xtest = torch.tensor(X_test_vec, dtype=torch.float32)
t_ytest = torch.tensor(y_test, dtype=torch.float32).reshape(-1, 1)

xtrain, xval, ytrain, yval = train_test_split(
    t_xtrain, t_ytrain, test_size=0.20, random_state=42
)

In [48]:
# compare performance across different model classes
model_lin_svm = SVC(
    kernel="linear",
    probability=True,
)
model_lin_svm.fit(X_train_vec, y_train)
print(f"linear SVM: {model_lin_svm.score(X_test_vec, y_test)}")

model_nn = MLPClassifier(
    solver="adam",
    alpha=1e-5,
    hidden_layer_sizes=(10, 5),
    max_iter=4000,
    early_stopping=True,
    random_state=1,
)

model_nn.fit(X_train_vec, y_train)
print(f"neural network 1: {model_nn.score(X_test_vec, y_test)}")

linear SVM: 0.7472527472527473
neural network 1: 0.7517776341305753


In [52]:
base_model

Wide(
  (hidden): Linear(in_features=10, out_features=10, bias=True)
  (relu): ReLU()
  (output): Linear(in_features=10, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)

In [49]:
dataset = eu.ExpDataset(t_xtrain, t_ytrain)

In [55]:
base_model = eu.Wide(input_size=t_xtrain.shape[1])
dataloader = DataLoader(dataset, batch_size=20, shuffle=True, num_workers=0)
acc = eu.model_train_base(base_model, dataloader, t_xtest, t_ytest)

100%|█████████████████████████████████████████████████████████████████████████████████| 50/50 [00:07<00:00,  7.07it/s]


In [59]:
def base_predict_proba_text(X_docs):
    X_vect = torch.tensor(tf_vec.transform(X_docs).todense(), dtype=torch.float32)
    preds = base_model(X_vect).detach().numpy()
    prob = np.concatenate((1 - preds, preds), axis=1)
    return prob


def aug_predict_proba_text(X_docs):
    X_vect = torch.tensor(tf_vec.transform(X_docs).todense(), dtype=torch.float32)
    preds = aug_model(X_vect).detach().numpy()
    prob = np.concatenate((1 - preds, preds), axis=1)
    return prob


def base_predict_proba_tab(X_docs):
    X_vect = torch.tensor(scale.transform(X_docs), dtype=torch.float32)
    preds = base_model(X_vect).detach().numpy()
    prob = np.concatenate((1 - preds, preds), axis=1)
    return prob


def aug_predict_proba_tab(X_docs):
    X_vect = torch.tensor(scale.transform(X_docs), dtype=torch.float32)
    preds = aug_model(X_vect).detach().numpy()
    prob = np.concatenate((1 - preds, preds), axis=1)
    return prob

In [79]:
aug_results = []
lr = 0.005
max_iter = 30

for ind in tqdm(range(len(y_test))):
    test_instance = t_xtest[ind]
    pred = base_model(test_instance).detach().round()

    X_test_aug = torch.vstack((t_xtest[:ind], t_xtest[ind + 1 :]))
    y_test_aug = torch.tensor(np.delete(y_test, ind), dtype=torch.float32).reshape(
        -1, 1
    )

    aug_model = eu.Wide(input_size=t_xtrain.shape[1])
    aug_model.load_state_dict(base_model.state_dict())

    # fine tune on 1 example
    optimizer = optim.Adam(aug_model.parameters(), lr=lr)
    loss_fn = nn.BCELoss()
    aug_model.train()
    for i in range(max_iter):
        y_pred = aug_model(test_instance)
        loss = loss_fn(y_pred, 1 - pred)
        # backward pass
        optimizer.zero_grad()
        loss.backward()
        # update weights
        optimizer.step()
        # print progress
        aug_model.eval()
        success = aug_model(test_instance).round() != pred
        if success:
            break

    y_pred = aug_model(X_test_aug)
    acc = (y_pred.round() == y_test_aug).float().mean()
    acc = float(acc)

    if DS == Data.REVIEWS:
        explainer = lime.lime_text.LimeTextExplainer(class_names=["deceptive", "real"])
        c1 = make_pipeline(tf_vec, model_lin_svm)
        aug_predict_proba = aug_predict_proba_text
        base_predict_proba = aug_predict_proba_text
        n_feat = 10
    else:
        explainer = lime.lime_tabular.LimeTabularExplainer(
            features,
            feature_names=ACSIncome._features,
            class_names=["<50k", "50k+"],
            discretize_continuous=False,
        )

        c1 = make_pipeline(scale, model_lin_svm)
        aug_predict_proba = aug_predict_proba_tab
        base_predict_proba = aug_predict_proba_tab
        n_feat = 5

    exp1 = explainer.explain_instance(
        X_test[ind], base_predict_proba, num_features=n_feat
    )

    exp1_feat = np.asarray([feat for feat, weight in exp1.local_exp[1]])
    exp1_weights = np.asarray([weight for feat, weight in exp1.local_exp[1]])

    exp2 = explainer.explain_instance(
        X_test[ind], aug_predict_proba, num_features=n_feat
    )

    exp2_feat = np.asarray([feat for feat, weight in exp2.local_exp[1]])
    exp2_weights = np.asarray([weight for feat, weight in exp2.local_exp[1]])

    aug_results.append(
        {
            "ind": ind,
            "orig_pred": pred.detach().numpy()[0],
            "retrain_acc": acc,
            "top_feat": eu.top_features(exp1_feat, exp2_feat),
            "pos_pred": eu.pos_features_agg(
                exp1_feat, exp1_weights, exp2_feat, exp2_weights
            ),
            "neg_pred": eu.neg_features_agg(
                exp1_feat, exp1_weights, exp2_feat, exp2_weights
            ),
            "success": success.numpy()[0],
            "iter": i,
        }
    )
    if ind % 10 == 0:
        model_aug_df = pd.DataFrame(aug_results)
        model_aug_df.to_csv(
            f"results/{dataset_str}_nn_aug_results_single_lr{lr}_max{max_iter}.csv"
        )

model_aug_df = pd.DataFrame(aug_results)
model_aug_df.to_csv(
    f"results/{dataset_str}_nn_aug_results_single_lr{lr}_max{max_iter}.csv"
)

100%|█████████████████████████████████████████████████████████████████████████████| 1547/1547 [05:24<00:00,  4.77it/s]


In [80]:
model_aug_df.groupby(
    pd.cut(model_aug_df["iter"], [0, 5, 10, 15, 20, 30])
).count() / len(y_test)

,ind,orig_pred,retrain_acc,top_feat,pos_pred,neg_pred,success,iter
iter,,,,,,,,
"(0, 5]",0.248222,0.248222,0.248222,0.248222,0.248222,0.248222,0.248222,0.248222
"(5, 10]",0.225598,0.225598,0.225598,0.225598,0.225598,0.225598,0.225598,0.225598
"(10, 15]",0.148675,0.148675,0.148675,0.148675,0.148675,0.148675,0.148675,0.148675
"(15, 20]",0.098255,0.098255,0.098255,0.098255,0.098255,0.098255,0.098255,0.098255
"(20, 30]",0.217195,0.217195,0.217195,0.217195,0.217195,0.217195,0.217195,0.217195


In [82]:
model_aug_df.groupby(pd.cut(model_aug_df["iter"], [0, 5, 10, 15, 20, 30])).std()

,ind,orig_pred,retrain_acc,top_feat,pos_pred,neg_pred,success,iter
iter,,,,,,,,
"(0, 5]",461.093267,0.500646,0.004175,0.080792,0.000000,0.161585,0.000000,1.442509
"(5, 10]",443.454619,0.491404,0.006075,0.094967,0.029751,0.183190,0.000000,1.424337
"(10, 15]",425.991517,0.480016,0.009064,0.089722,0.032752,0.177144,0.000000,1.339396
"(15, 20]",441.944990,0.365848,0.013015,0.074678,0.028582,0.141004,0.000000,1.437376
"(20, 30]",441.460598,0.281239,0.019071,0.106929,0.047696,0.207334,0.494713,2.857699
